# Multi-Agent Hackathon Scenario with AutoGen and Ollama Phi3

This notebook demonstrates how to use [AutoGen](https://github.com/microsoft/autogen) with the Ollama Phi3 model to coordinate a team of specialized agents for a hackathon project. Each agent represents a different role, collaborating to take a project idea from concept to delivery.

## Scenario Overview

- **Planner (Project Manager):** Expands the project idea into epics, features, stories, and tasks with estimates.
- **UX Engineer:** Plans wireframes, screens, and Figma prompts.
- **Reviewer:** Evaluates plans for security, customer value, and workflow improvements.
- **Backend Developer:** Designs APIs, FastAPI snippets, and DynamoDB diagrams.
- **Frontend Developer:** Plans React structure, components, and dependencies.
- **Marketing Agent:** Prepares slides, executive summary, and patent claims.
- **Data Scientist/AI Engineer:** Designs ML models, LLM prompts, and AIops plans.
- **DevOps Engineer:** Sets up CI/CD, Dockerfiles, Terraform, and monitoring.
- **Technical Writer:** Documents architecture, diagrams, and use cases.

The agents interact in a workflow to deliver a comprehensive hackathon project plan.


#fetch https://github.com/LinkedInLearning/building-ai-agents-with-autogen-3918551/blob/04_02e/03_03_e.ipynb

#fetch https://github.com/LinkedInLearning/building-ai-agents-with-autogen-3918551/blob/04_02e/03_02_e.ipynb

In [2]:
# Install required packages (if not already installed)
# !pip install pyautogen
#import openai
import autogen

In [3]:
# Set up the Ollama Phi3 model for use with AutoGen
OLLAMA_BASE_URL = "http://localhost:11434"  # Change if your Ollama server runs elsewhere
MODEL_NAME = "phi3"
ollama_model = "phi3"
# Example: ollama.pull(ollama_model)  # Ensure the model is available locally

llm_config = {
    'base_url': OLLAMA_BASE_URL,
    'api_key': "NULL",
    'model': MODEL_NAME
    # "model": ollama_model,
    # "temperature": 0.2,
    # "max_tokens": 2048,
}

## Test Autogen

In [5]:
from autogen import AssistantAgent

# Setup API key. Add your own API key to config file or environment variable
config_list = [
    {
        "model": "phi3",
        "base_url": "http://localhost:11434/v1",
        'api_key': 'ollama',
    },
]

small = AssistantAgent(name="smallmodel",
                       max_consecutive_auto_reply=2,
                       system_message="You should act as a student!",
                       llm_config={
                           "config_list": config_list,
                           "temperature": 1,
                       })

big = AssistantAgent(name="bigmodel",
                     max_consecutive_auto_reply=2,
                     system_message="Act as a teacher.",
                     llm_config={
                         "config_list": config_list,
                         "temperature": 1,
                     })
big.initiate_chat(small, message="Who are you?")

bigmodel (to smallmodel):

Who are you?

--------------------------------------------------------------------------------
[autogen.oai.client: 05-31 19:14:08] {696} WARNING - Model phi3 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
smallmodel (to bigmodel):

Hello, I am an artificial intelligence designed to help users with various queries and tasks. My purpose is to provide accurate information, facilitate problem-solving, or engage in friendly conversation based on algorithms processed by my programming team. How can I assist you today?

--------------------------------------------------------------------------------
[autogen.oai.client: 05-31 19:14:09] {696} WARNING - Model phi3 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
bigmodel (to smallmodel):

I appreciate the int

KeyboardInterrupt: 

In [13]:
# Define agent roles and their responsibilities

agent_configs = [
    {"name": "Planner", "role": "Expands project idea into epics, features, stories, and tasks with estimates."},
    {"name": "UXEngineer", "role": "Plans wireframes, screens, Figma prompts."},
    {"name": "Reviewer", "role": "Evaluates plans for security, customer value, and workflow improvements."},
    {"name": "BackendDev", "role": "Designs APIs, FastAPI snippets, DynamoDB diagrams."},
    {"name": "FrontendDev", "role": "Plans React structure, components, and dependencies."},
    {"name": "Marketing", "role": "Prepares slides, executive summary, and patent claims."},
    {"name": "DataScientist", "role": "Designs ML models, LLM prompts, and AIops plans."},
    {"name": "DevOps", "role": "Sets up CI/CD, Dockerfiles, Terraform, and monitoring."},
    {"name": "TechWriter", "role": "Documents architecture, diagrams, and use cases."},
]

In [14]:
# Create AutoGen agents for each role

agents = {}
for config in agent_configs:
    agent = autogen.AssistantAgent(
        name=config["name"],
        llm_config=llm_config,
        system_message=config["role"]
    )
    agents[config["name"]] = agent

## Example Workflow

Let's walk through a sample workflow where the agents collaborate on a hackathon project idea.

In [16]:
# Example Workflow using AssistantAgent and initiate_chat

from autogen import AssistantAgent

# Setup config_list for all agents
config_list = [
    {
        "model": "phi3",
        "base_url": "http://localhost:11434/v1",
        'api_key': 'ollama',
    },
]

# Create agents for each role
agents = {}
for config in agent_configs:
    agent = AssistantAgent(
        name=config["name"],
        max_consecutive_auto_reply=2,
        system_message=config["role"],
        llm_config={
            "config_list": config_list,
            "temperature": 1,
        }
    )
    agents[config["name"]] = agent

# Define the project idea
project_idea = """
AI-powered Threat Detection Dashboard: 
A platform that helps organizations identify, visualize, and respond to security threats in real time using AI and data analytics.
"""

# Planner and Reviewer interaction
planner = agents["Planner"]
reviewer = agents["Reviewer"]
ux = agents["UXEngineer"]
backend = agents["BackendDev"]
frontend = agents["FrontendDev"]
datasci = agents["DataScientist"]
devops = agents["DevOps"]
marketing = agents["Marketing"]
techwriter = agents["TechWriter"]

# Step 1: Planner expands the idea, Reviewer reviews
planner.initiate_chat(
    reviewer,
    message=project_idea + "\nExpand this idea into epics, features, stories, and tasks with estimated days for each task. Then review for security, customer value, and workflow improvement."
)

# Step 2: UX Engineer plans wireframes and Figma prompts with Planner
planner.initiate_chat(
    ux,
    message="Based on the expanded plan, list all screens, interactions, and Figma AI prompts needed."
)

# Step 3: Backend, Frontend, and Data Scientist work in parallel with UX Engineer
ux.initiate_chat(
    backend,
    message="Design APIs, provide FastAPI code snippets, and DynamoDB (PlantUML) diagram for the planned screens."
)
ux.initiate_chat(
    frontend,
    message="Plan React project folder structure, components, and package.json for the planned screens."
)
ux.initiate_chat(
    datasci,
    message="Suggest ML models, LLM prompts, and AIops steps for the project."
)

# Step 4: DevOps Engineer plans deployment and monitoring with Backend
backend.initiate_chat(
    devops,
    message="Provide Jenkinsfile for CI/CD, Dockerfiles for frontend/backend/AI, Terraform for architecture, and a monitoring/alerting plan."
)

# Step 5: Marketing Agent prepares slides, summary, and patent claims with Planner
planner.initiate_chat(
    marketing,
    message="Prepare Reveal.js slides, executive summary, patent claims, use cases, pain points, and a 3-minute video script."
)

# Step 6: Technical Writer documents the project with Marketing
marketing.initiate_chat(
    techwriter,
    message="Write abstract, description, detailed description, PlantUML diagrams (overall and deep dive), and use case workflow diagram."
)

Planner (to Reviewer):


AI-powered Threat Detection Dashboard: 
A platform that helps organizations identify, visualize, and respond to security threats in real time using AI and data analytics.

Expand this idea into epics, features, stories, and tasks with estimated days for each task. Then review for security, customer value, and workflow improvement.

--------------------------------------------------------------------------------
[autogen.oai.client: 05-31 19:24:43] {696} WARNING - Model phi3 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
Reviewer (to Planner):

Epic: Enhance Security Threat Detection Capabilities  
Features (all are ongoing):    
- Collect real-time traffic data from network devices using integrated API hooks into systems like Cisco IOS, Juniper Junos and others. Estimated completion time for team research is approximately 5 days to choose appropriate APIs 

ChatResult(chat_id=None, chat_history=[{'content': 'Write abstract, description, detailed description, PlantUML diagrams (overall and deep dive), and use case workflow diagram.', 'role': 'assistant', 'name': 'Marketing'}, {'content': '            From a theoretical scenario wherein the given text to be used as an expert hygiene is required for each step in question: <|endofindings**/Input-aware_hcDellinger GmbH, \n \n### I\'m sorry I apologize! That’deas and understanding of this document. In a new religious studies class taught that you have been provided below are the following three tables detailing information about an imaginary landmark in Algeria where we can determine if they satisfy certain criteria for all given entities, then please answer some general relativity theories \ncustomer:\n\n- GASB\'s response. Andresendex each year of herbaceous plants and the rest are toxic substances is $30164 in a specific sequence from various types of natural remedies for anxiety, like this?